# Kumru - Fincan Kahvem Hatrina
## AI Rap Song Generation Pipeline (Colab Edition)

Bu notebook, **Kumru-2B** base model + **LoRA adapter** (Sagopa Kajmer & Ceza) kullanarak
tam otonom sarki uretim pipeline'ini calistirir.

### Gereksinimler
- Colab'a yuklenmus `CezaWeights/` ve `SagoWeights/` klasorleri
- `.env` dosyasi (OpenRouter API key icin)
- **GPU runtime** (T4 yeterli, 2B model ~4GB VRAM)

## 0. GPU Kontrol

In [ ]:
!nvidia-smi
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")

## 1. Kutuphane Kurulumu

In [ ]:
!pip install -q transformers>=4.36.0 peft>=0.18.0 accelerate>=0.25.0 \
    bitsandbytes>=0.41.0 safetensors>=0.4.0 \
    langgraph>=0.2.0 langchain-core>=0.3.0 \
    httpx>=0.25.0 python-dotenv>=1.0.0 pydantic>=2.0.0 \
    nest_asyncio>=1.5.0

## 2. Dosyalari Yukle

Colab'a su klasorleri yuklemis olmaniz gerekiyor:
- `CezaWeights/` (adapter_config.json, adapter_model.safetensors, tokenizer dosyalari)
- `SagoWeights/` (adapter_config.json, adapter_model.safetensors, tokenizer dosyalari)
- `.env` dosyasi

Eger Google Drive'dan yukleyecekseniz asagidaki cell'i kullanin:

In [ ]:
# ========================================
# YONTEM 1: Google Drive'dan yukle
# ========================================
# Drive'iniza CezaWeights, SagoWeights ve .env dosyasini yukleyin
# ve asagidaki path'i guncelleyin

from google.colab import drive
drive.mount('/content/drive')

# Drive'daki klasor yolunu buraya yazin
DRIVE_PROJECT_PATH = "/content/drive/MyDrive/FincanKahvemHatrina"  # <-- BUNU GUNCELLEYIN

import shutil, os

for folder in ["CezaWeights", "SagoWeights"]:
    src = os.path.join(DRIVE_PROJECT_PATH, folder)
    dst = f"/content/{folder}"
    if os.path.exists(src) and not os.path.exists(dst):
        shutil.copytree(src, dst)
        print(f"Kopyalandi: {src} -> {dst}")
    elif os.path.exists(dst):
        print(f"Zaten mevcut: {dst}")
    else:
        print(f"BULUNAMADI: {src}")

env_src = os.path.join(DRIVE_PROJECT_PATH, ".env")
if os.path.exists(env_src):
    shutil.copy2(env_src, "/content/.env")
    print(f"Kopyalandi: {env_src} -> /content/.env")
else:
    print(f"BULUNAMADI: {env_src}")

In [ ]:
# ========================================
# YONTEM 2: Colab'a elle yukleme (upload)
# ========================================
# Sidebar'dan dosyalari suruklemeniz yeterli.
# Asagidaki cell dosyalarin varligini kontrol eder.

import os

required = [
    "/content/CezaWeights/adapter_config.json",
    "/content/CezaWeights/adapter_model.safetensors",
    "/content/SagoWeights/adapter_config.json",
    "/content/SagoWeights/adapter_model.safetensors",
    "/content/.env",
]

all_ok = True
for path in required:
    exists = os.path.exists(path)
    status = "OK" if exists else "EKSIK"
    print(f"  [{status}] {path}")
    if not exists:
        all_ok = False

if all_ok:
    print("\nTum dosyalar mevcut!")
else:
    print("\nEKSIK dosyalar var! Lutfen yukleyin.")

## 3. Environment Variables (.env)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/content/.env")

# Colab icin: Lokal LoRA kullan (RunPod degil)
os.environ["USE_LOCAL_MODELS"] = "true"
os.environ["SAGOPA_LORA_PATH"] = "/content/SagoWeights"
os.environ["CEZA_LORA_PATH"] = "/content/CezaWeights"

print("Environment degiskenleri:")
print(f"  USE_LOCAL_MODELS  = {os.getenv('USE_LOCAL_MODELS')}")
print(f"  SAGOPA_LORA_PATH  = {os.getenv('SAGOPA_LORA_PATH')}")
print(f"  CEZA_LORA_PATH    = {os.getenv('CEZA_LORA_PATH')}")
print(f"  OPENAI_API_KEY    = {os.getenv('OPENAI_API_KEY', '')[:20]}...")
print(f"  OPENROUTHER_MODEL = {os.getenv('OPENROUTHER_MODEL')}")

## 4. Kumru-2B Base Model + LoRA Yukleme

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
log = logging.getLogger("kumru_colab")

BASE_MODEL_NAME = "vngrs-ai/Kumru-2B"
SAGOPA_LORA_PATH = "/content/SagoWeights"
CEZA_LORA_PATH = "/content/CezaWeights"

# Base model yukle
print("Base model yukleniyor: Kumru-2B ...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
)

model_size_gb = sum(p.numel() * p.element_size() for p in base_model.parameters()) / 1024**3
print(f"Base model yuklendi! (~{model_size_gb:.2f} GB)")
print(f"Device: {base_model.device}")

# LoRA adapter'larini yukle
print("\nSagopa LoRA yukleniyor...")
sagopa_model = PeftModel.from_pretrained(base_model, SAGOPA_LORA_PATH)
print("Sagopa LoRA yuklendi!")

print("\nCeza LoRA yukleniyor...")
ceza_model = PeftModel.from_pretrained(base_model, CEZA_LORA_PATH)
print("Ceza LoRA yuklendi!")

print("\nTum modeller hazir!")

## 5. Inference Fonksiyonlari

In [ ]:
import re

def format_chatml_prompt(
    instruction: str,
    input_text: str,
    artist: str = "sagopa",
    system_prompt: str = "",
) -> str:
    """
    Kumru-2B ChatML formatinda prompt olustur.

    ChatML yapisi:
        <|im_start|>system\n{system}\n<|im_end|>
        <|im_start|>user\n{instruction}\n{input}\n<|im_end|>
        <|im_start|>assistant\n

    Egitim dataseti formati:
        instruction = "Asagidaki temada {Artist} tarzi bir rap verse'u yaz."
        input       = "{tema/konu}"
        output      = "{sarki sozu}"
    """
    # Varsayilan instruction (dataset formatiyla ayni)
    if not instruction:
        if artist.lower() == "sagopa":
            instruction = "Asagidaki temada Sagopa Kajmer tarzi bir rap verse'u yaz."
        elif artist.lower() == "ceza":
            instruction = "Asagidaki temada Ceza tarzi bir rap verse'u yaz."

    parts = []

    # System mesaji (varsa)
    if system_prompt:
        parts.append(f"<|im_start|>system\n{system_prompt}<|im_end|>")

    # User mesaji: instruction + input
    parts.append(
        f"<|im_start|>user\n"
        f"{instruction}\n"
        f"{input_text}<|im_end|>"
    )

    # Assistant baslangici (model buradan devam edecek)
    parts.append("<|im_start|>assistant\n")

    return "\n".join(parts)


def clean_special_tokens(text: str) -> str:
    """LoRA model output'larindan special token'lari ve copu temizler."""
    text = re.sub(r"<\|im_start\|>.*?(?=\n|$)", "", text)
    text = re.sub(r"<\|im_end\|>", "", text)
    text = re.sub(r"<\|.*?\|>", "", text)

    meta_patterns = [
        r"^(?:Sozler|Ask Sozleri|Ruh Hali Teknikleri|Akor Tekni[g\u011f]i|"
        r"Verse|Mood|Tema|Yonerge|Not|Review notu|Son Tahlilde|"
        r"Vokal vurgusu|Downbeat|Rhyme):.*$",
        r"^\(Vokal vurgusu:.*?\)$",
        r"^\u2b50.*$",
        r"^\u2600\ufe0f.*$",
        r"^---+$",
        r"^#+\s.*$",
        r"^\*\*.*\*\*:?$",
        r'^_\".*\"_+_*$',
    ]
    for pattern in meta_patterns:
        text = re.sub(pattern, "", text, flags=re.MULTILINE)

    lines = text.split("\n")
    cleaned_lines = []
    prev_line = None
    repeat_count = 0
    for line in lines:
        stripped = line.strip()
        if stripped == prev_line and stripped:
            repeat_count += 1
            if repeat_count >= 2:
                continue
        else:
            repeat_count = 0
        cleaned_lines.append(line)
        prev_line = stripped
    text = "\n".join(cleaned_lines)

    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def generate_verse(prompt: str, artist: str = "sagopa",
                   instruction: str = "",
                   system_prompt: str = "",
                   max_new_tokens: int = 512,
                   temperature: float = 0.8,
                   top_p: float = 0.9,
                   repetition_penalty: float = 1.3) -> str:
    """
    Verilen artist LoRA modeli ile verse uret.

    Args:
        prompt: Tema/konu metni (ChatML user mesajindaki input)
        artist: 'sagopa' veya 'ceza'
        instruction: User mesajindaki instruction (bos ise varsayilan kullanilir)
        system_prompt: ChatML system mesaji (opsiyonel, stil profili icin)
        max_new_tokens: Uretilecek max token
        temperature: Sampling sicakligi
        top_p: Nucleus sampling p
        repetition_penalty: Tekrar cezasi
    """
    model = sagopa_model if artist.lower() == "sagopa" else ceza_model

    formatted_prompt = format_chatml_prompt(
        instruction=instruction,
        input_text=prompt,
        artist=artist,
        system_prompt=system_prompt,
    )

    inputs = tokenizer(formatted_prompt, return_tensors="pt")
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]
    model_inputs = {k: v.to(model.device) for k, v in inputs.items()}
    input_len = model_inputs["input_ids"].shape[1]

    with torch.no_grad():
        model_outputs = model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            pad_token_id=tokenizer.eos_token_id,
        )

    generated_ids = model_outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=False)
    if "<|im_end|>" in response:
        response = response.split("<|im_end|>")[0]

    return clean_special_tokens(response.strip())


print("Inference fonksiyonlari hazir!")

## 6. Hizli Test - Tek Verse Uretimi

In [ ]:
# Sagopa test
print("=" * 60)
print("SAGOPA KAJMER TEST")
print("=" * 60)
sagopa_verse = generate_verse(
    prompt="Hayatin anlami ve melankolik yalnizlik",
    artist="sagopa",
    max_new_tokens=256,
)
print(sagopa_verse)

print()

# Ceza test
print("=" * 60)
print("CEZA TEST")
print("=" * 60)
ceza_verse = generate_verse(
    prompt="Sokak hayati ve mucadele",
    artist="ceza",
    max_new_tokens=256,
)
print(ceza_verse)

## 7. LangGraph Pipeline - Lokal LoRA Endpoint Tanimi

Pipeline'in Colab'da calismasi icin `LocalLoRAEndpoint`'i tanimliyoruz.
Bu endpoint, yukarida yukledigimiz modelleri dogrudan kullaniyor.

In [ ]:
import asyncio
import json
import random
import nest_asyncio
from datetime import datetime
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
import httpx
import threading

nest_asyncio.apply()

# ============================================================
# SHARED MEMORY - Paralel node'lar arasi canli iletisim
# ============================================================
# LangGraph paralel node'lari state'i paylasamaz (her biri kendi kopyasiyla calisir).
# Bu global dict, sagopa ve ceza node'larinin birbirlerinin verse'lerini
# yazim sirasinda canli olarak gormesini saglar.

class SharedMemory:
    """Thread-safe shared memory for live cross-artist awareness."""
    def __init__(self):
        self._lock = threading.Lock()
        self._verses = {}  # {"sagopa_v1": "...", "ceza_v1": "...", ...}

    def write(self, section: str, verse: str):
        """Verse yazildiginda memory'ye kaydet."""
        with self._lock:
            self._verses[section] = verse
            print(f"  [SHARED_MEMORY] {section} yazildi ({len(verse)} kar)")

    def read_other(self, artist: str) -> str:
        """Diger sanatcinin su ana kadar yazdigi verse'leri oku."""
        with self._lock:
            parts = []
            for section, verse in self._verses.items():
                # Sadece diger sanatcinin verse'lerini dondur
                if artist == "sagopa" and section.startswith("ceza_"):
                    parts.append(f"[{section}]: {verse[:300]}")
                elif artist == "ceza" and section.startswith("sagopa_"):
                    parts.append(f"[{section}]: {verse[:300]}")
            return "\n".join(parts)

    def read_all(self) -> str:
        """Tum verse'leri oku (critic/hook icin)."""
        with self._lock:
            parts = []
            for section, verse in sorted(self._verses.items()):
                parts.append(f"[{section}]: {verse[:300]}")
            return "\n".join(parts)

    def clear(self, keep_sections: list = None):
        """Yeni iterasyon icin temizle. keep_sections verilirse sadece onlari korur."""
        with self._lock:
            if keep_sections:
                kept = {k: v for k, v in self._verses.items() if k in keep_sections}
                removed = [k for k in self._verses if k not in keep_sections]
                self._verses = kept
                print(f"  [SHARED_MEMORY] Temizlendi (korunan: {list(kept.keys())}, silinen: {removed})")
            else:
                self._verses.clear()
                print("  [SHARED_MEMORY] Tamamen temizlendi")

shared_memory = SharedMemory()


# ============================================================
# STATE DEFINITION
# ============================================================

class SongState(TypedDict):
    theme: str
    mood: str
    additional_directives: str
    sagopa_style_prompt: str
    ceza_style_prompt: str
    song_structure: list[str]
    section_briefs: dict
    sagopa_verses: list[str]
    ceza_verses: list[str]
    hook: str
    critic_scores: dict
    critic_feedback: dict
    revision_count: int
    hook_revision_count: int
    hook_critic_score: float
    hook_critic_feedback: str
    song_title: str
    final_song: str
    beat_suggestion: dict
    status: str


# ============================================================
# ENDPOINT CLIENTS
# ============================================================

class OpenAIEndpoint:
    """OpenAI-compatible endpoint (OpenRouter)."""
    def __init__(self, base_url: str, model_name: str, api_key: str = ""):
        self.base_url = base_url.rstrip("/")
        self.model_name = model_name
        self.headers = {"Content-Type": "application/json"}
        if api_key:
            self.headers["Authorization"] = f"Bearer {api_key}"

    async def generate(self, prompt: str, system_prompt: str = "", max_tokens: int = 1024) -> str:
        payload = {
            "model": self.model_name,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
            "max_tokens": max_tokens,
            "temperature": 0.8,
            "top_p": 0.9,
        }
        url = f"{self.base_url}/v1/chat/completions"
        print(f"  >>> [{self.model_name}] istek -> {url}")
        async with httpx.AsyncClient(timeout=60.0) as client:
            response = await client.post(url, json=payload, headers=self.headers)
            response.raise_for_status()
            data = response.json()
            content = data["choices"][0]["message"]["content"]
            print(f"  <<< [{self.model_name}] cevap ({len(content)} kar)")
            return content


class ColabLoRAEndpoint:
    """Colab'da yuklenmis LoRA modelleri ile dogrudan inference."""
    def __init__(self, artist: str):
        self.artist = artist
        self.model_name = f"{artist}-lora-colab"

    async def generate(self, prompt: str, system_prompt: str = "", max_tokens: int = 512) -> str:
        print(f"  >>> [{self.model_name}] Lokal LoRA inference basliyor...")
        loop = asyncio.get_event_loop()
        result = await loop.run_in_executor(
            None,
            lambda: generate_verse(
                prompt=prompt,
                artist=self.artist,
                instruction=system_prompt,
                system_prompt="",
                max_new_tokens=max_tokens,
                temperature=0.8,
                top_p=0.9,
                repetition_penalty=1.3,
            )
        )
        print(f"  <<< [{self.model_name}] cevap ({len(result)} kar)")
        return result


# Endpoint'leri olustur
sagopa_endpoint = ColabLoRAEndpoint(artist="sagopa")
ceza_endpoint = ColabLoRAEndpoint(artist="ceza")

orchestrator_endpoint = OpenAIEndpoint(
    base_url="https://openrouter.ai/api",
    model_name=os.getenv("OPENROUTHER_MODEL", "anthropic/claude-haiku-4.5"),
    api_key=os.getenv("OPENAI_API_KEY", ""),
)

print("Endpoint'ler ve Shared Memory hazir!")
print(f"  Sagopa: ColabLoRAEndpoint (GPU uzerinde)")
print(f"  Ceza:   ColabLoRAEndpoint (GPU uzerinde)")
print(f"  Orchestrator: OpenRouter ({os.getenv('OPENROUTHER_MODEL', 'anthropic/claude-haiku-4.5')})")
print(f"  SharedMemory: Aktif (thread-safe)")

## 8. Pipeline Node Fonksiyonlari

In [ ]:
# ============================================================
# HELPER
# ============================================================

def extract_json(text: str):
    """Markdown code block veya duz JSON'dan dict parse eder."""
    match = re.search(r"```(?:json)?\s*\n?(.*?)```", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(1).strip())
        except json.JSONDecodeError:
            pass
    try:
        return json.loads(text.strip())
    except json.JSONDecodeError:
        pass
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except json.JSONDecodeError:
            pass
    return None


def safe_score(raw_score) -> float:
    """Critic skorunu guvenli sekilde float'a cevir.
    Haiku bazen 7.2 yerine 72, 6.5 yerine 65 dondurebiliyor.
    10'dan buyuk skorlari 10'a boler."""
    try:
        s = float(raw_score)
        if s > 10:
            s = s / 10.0
        return round(s, 1)
    except (TypeError, ValueError):
        return 5.0


def is_verse_empty(text: str) -> bool:
    cleaned = clean_special_tokens(text)
    if not cleaned or len(cleaned) < 20:
        return True
    return False


# ============================================================
# SYSTEM PROMPTS
# ============================================================

IDEATION_SYSTEM = """Sen bir sarki konsept gelistiricisisin.
Gorev: Iki sanatcinin birlikte yapacagi bir sarki icin orijinal bir konsept olustur.

SADECE JSON dondur, baska hicbir sey yazma:
{"theme": "...", "mood": "...", "directives": "..."}
"""

STYLE_PROFILER_SYSTEM = """Sen bir sanatci stil analisti ve profil olusturucususun.
Gorevin: Verilen sanatcilar icin, sarki sozleri yazarken kullanilacak stil profilleri olusturmak.

SADECE JSON dondur:
{"sagopa_style_prompt": "...", "ceza_style_prompt": "..."}
"""

PRODUCER_SYSTEM = """Sen bir sarki produktorusun.
Gorevin verilen tema ve mood'a gore sarki yapisini planlamak.
Bolum isimleri icin sadece su key'leri kullan: intro, sagopa_v1, sagopa_v2, ceza_v1, ceza_v2, hook, hook_repeat, outro.

SADECE JSON dondur:
{"structure": ["bolum1", ...], "briefs": {"bolum1": "kisa yonerge", ...}}
"""

# Critic'e ACIK talimat: score 1-10 arasi ONDALIKLI sayi olmali
CRITIC_SYSTEM = """Sen bir sarki sozu elestirmenisin.
Gorevin verilen verse'i degerlendirmek. Degerlendirme kriterleri:
1. Sanatcinin kendi stiline uygunluk (flow, kelime secimi, karakteristik ifadeler)
2. Diger sanatcinin verse'leriyle tematik uyum ve birbirini tamamlama
3. Lirik kalite, metafor kullanimi, akicilik
4. Sarkinin genel temasina ve mood'una baglilik

Onemli: Iki sanatcinin verse'leri bir butun olarak uyumlu olmali. Biri digerini tamamlamali.
Bu sanatcilar kucuk bir dil modeliyle (2B parametre) uretilmis. Mukemmeli bekleme ama temel kaliteyi ara.

SADECE JSON dondur. score 1 ile 10 arasi bir ONDALIKLI sayi olmali (ornek: 6.5, 7.0, 8.2).
feedback kisaca ve net olmali (1-2 cumle, Turkce).
{"score": 7.0, "feedback": "...", "approved": true}
"""

HOOK_CRITIC_SYSTEM = """Sen bir nakarat elestirmenisin.
Gorevin verilen nakarati sarkinin geneli ile uyum acisindan degerlendirmek.
Degerlendirme kriterleri:
1. Verse'lerle tematik uyum - nakarat sarkinin verse'leriyle butunluk saglamali
2. Akilda kalicilik - tekrar dinlenebilir, melodik bir yapi
3. Iki sanatcinin stilini birlestirme basarisi
4. Sarkinin mood'una uygunluk

SADECE JSON dondur. score 1 ile 10 arasi bir ONDALIKLI sayi olmali (ornek: 6.5, 7.0, 8.2).
{"score": 7.0, "feedback": "...", "approved": true}
"""

TITLE_SYSTEM = """Sen bir sarki isimlendirme uzmanisin.
Verilen sarki sozlerine bakarak sarkiya uygun, akilda kalici bir isim sec.
Sadece ismi dondur, baska hicbir sey yazma.
"""

# Esik degeri: 2B model icin 6.5 daha gercekci
VERSE_PASS_THRESHOLD = 6.5
HOOK_PASS_THRESHOLD = 6.5


# ============================================================
# NODE FUNCTIONS
# ============================================================

async def ideation_node(state: SongState) -> dict:
    print("\n" + "=" * 60)
    print("ADIM 0 - IDEATION: Tema/mood/directive uretiliyor...")
    print("=" * 60)
    seed = random.randint(1, 10000)
    prompt = f"Iki sanatci icin orijinal bir sarki konsepti olustur. Yaratici ol, klise temalardan kacin. (seed: {seed})"
    response = await orchestrator_endpoint.generate(prompt, IDEATION_SYSTEM)
    idea = extract_json(response)
    if not idea:
        response = await orchestrator_endpoint.generate(prompt, IDEATION_SYSTEM)
        idea = extract_json(response)
        if not idea:
            raise RuntimeError("Ideation JSON uretemedi.")
    print(f"  Tema: {idea.get('theme', '')}")
    print(f"  Mood: {idea.get('mood', '')}")
    print(f"  Direktifler: {idea.get('directives', '')}")
    return {"theme": idea["theme"], "mood": idea["mood"], "additional_directives": idea.get("directives", "")}


async def style_profiler_node(state: SongState) -> dict:
    print("\n" + "=" * 60)
    print("ADIM 0.5 - STYLE PROFILER")
    print("=" * 60)
    prompt = f"Sarki temasi: {state['theme']}\nMood: {state['mood']}\nEk yonergeler: {state.get('additional_directives', '')}\n\nSagopa Kajmer ve Ceza icin stil profilleri olustur."
    response = await orchestrator_endpoint.generate(prompt, STYLE_PROFILER_SYSTEM)
    profiles = extract_json(response)
    if not profiles:
        response = await orchestrator_endpoint.generate(prompt, STYLE_PROFILER_SYSTEM)
        profiles = extract_json(response)
        if not profiles:
            raise RuntimeError("Style profiler JSON uretemedi.")
    print(f"  Sagopa prompt: {profiles.get('sagopa_style_prompt', '')[:100]}...")
    print(f"  Ceza prompt:   {profiles.get('ceza_style_prompt', '')[:100]}...")
    return {"sagopa_style_prompt": profiles["sagopa_style_prompt"], "ceza_style_prompt": profiles["ceza_style_prompt"]}


async def producer_node(state: SongState) -> dict:
    print("\n" + "=" * 60)
    print(f"ADIM 1 - PRODUCER: Sarki yapisi planlaniyor... (Tema: {state['theme']})")
    print("=" * 60)
    prompt = f"Tema: {state['theme']}\nMood: {state['mood']}\nEk Yonergeler: {state.get('additional_directives', 'Yok')}\n\nSarki yapisini planla."
    response = await orchestrator_endpoint.generate(prompt, PRODUCER_SYSTEM, max_tokens=2048)
    plan = extract_json(response)
    if not plan:
        response = await orchestrator_endpoint.generate(prompt, PRODUCER_SYSTEM, max_tokens=2048)
        plan = extract_json(response)
        if not plan:
            raise RuntimeError("Producer JSON uretemedi.")
    structure = plan.get("structure", [])
    briefs = plan.get("briefs", {})
    print(f"  Yapi: {' -> '.join(structure)}")
    for sec, brief in briefs.items():
        print(f"    [{sec}] {brief}")
    shared_memory.clear()
    return {"song_structure": structure, "section_briefs": briefs, "status": "planning_complete"}


async def sagopa_node(state: SongState) -> dict:
    """Sagopa verse'lerini yazar. Shared memory uzerinden Ceza'yi canli takip eder."""
    print("\n" + "=" * 60)
    print(f"ADIM 2a - SAGOPA KAJMER: Verse yaziliyor... (iterasyon {state.get('revision_count', 0)})")
    print("=" * 60)
    briefs = state.get("section_briefs", {})
    verses = []

    for section in state.get("song_structure", []):
        if section.startswith("sagopa_"):
            brief = briefs.get(section, state["theme"])
            feedback = state.get("critic_feedback", {}).get(section, "")

            # --- SHARED MEMORY: Ceza'nin su ana kadar yazdiklarina bak ---
            ceza_context = shared_memory.read_other("sagopa")

            # Prompt'u KISA tut - 2B model uzun prompt'larla bozuluyor
            theme_input = f"{state['theme']}\n{brief}"

            if ceza_context:
                theme_input += f"\n\nCeza'nin verse'leri:\n{ceza_context}"
                print(f"  [{section}] SHARED MEMORY'den Ceza context okudu")

            if feedback:
                theme_input += f"\n\n(Not: {feedback[:100]})"

            instruction = "Asagidaki temada Sagopa Kajmer tarzi bir rap verse'u yaz."
            print(f"  [{section}] icin uretiliyor...")
            raw_verse = await sagopa_endpoint.generate(theme_input, instruction, max_tokens=512)
            verse = clean_special_tokens(raw_verse) if isinstance(raw_verse, str) else raw_verse
            if is_verse_empty(verse):
                raw_verse = await sagopa_endpoint.generate(theme_input, instruction, max_tokens=512)
                verse = clean_special_tokens(raw_verse)
                if is_verse_empty(verse):
                    verse = f"[{section} - verse uretilemedi]"
            print(f"  [{section}] Sagopa verse:\n{verse[:200]}...")

            shared_memory.write(section, verse)
            verses.append(verse)
    return {"sagopa_verses": verses}


async def ceza_node(state: SongState) -> dict:
    """Ceza verse'lerini yazar. Shared memory uzerinden Sagopa'yi canli takip eder."""
    print("\n" + "=" * 60)
    print(f"ADIM 2b - CEZA: Verse yaziliyor... (iterasyon {state.get('revision_count', 0)})")
    print("=" * 60)
    briefs = state.get("section_briefs", {})
    verses = []

    for section in state.get("song_structure", []):
        if section.startswith("ceza_"):
            brief = briefs.get(section, state["theme"])
            feedback = state.get("critic_feedback", {}).get(section, "")

            # --- SHARED MEMORY: Sagopa'nin su ana kadar yazdiklarina bak ---
            sagopa_context = shared_memory.read_other("ceza")

            # Prompt'u KISA tut
            theme_input = f"{state['theme']}\n{brief}"

            if sagopa_context:
                theme_input += f"\n\nSagopa'nin verse'leri:\n{sagopa_context}"
                print(f"  [{section}] SHARED MEMORY'den Sagopa context okudu")

            if feedback:
                theme_input += f"\n\n(Not: {feedback[:100]})"

            instruction = "Asagidaki temada Ceza tarzi bir rap verse'u yaz."
            print(f"  [{section}] icin uretiliyor...")
            raw_verse = await ceza_endpoint.generate(theme_input, instruction, max_tokens=512)
            verse = clean_special_tokens(raw_verse) if isinstance(raw_verse, str) else raw_verse
            if is_verse_empty(verse):
                raw_verse = await ceza_endpoint.generate(theme_input, instruction, max_tokens=512)
                verse = clean_special_tokens(raw_verse)
                if is_verse_empty(verse):
                    verse = f"[{section} - verse uretilemedi]"
            print(f"  [{section}] Ceza verse:\n{verse[:200]}...")

            shared_memory.write(section, verse)
            verses.append(verse)
    return {"ceza_verses": verses}


async def hook_node(state: SongState) -> dict:
    """Nakarat yazar. Verse'ler onaylandiktan sonra calisir."""
    hook_rev = state.get("hook_revision_count", 0)
    print("\n" + "=" * 60)
    print(f"ADIM 3 - HOOK: Nakarat yaziliyor... (hook iterasyon {hook_rev})")
    print("=" * 60)

    prev_feedback = state.get("hook_critic_feedback", "")

    prompt = f"""Tema: {state['theme']}
Mood: {state['mood']}

Sagopa'nin verse'i (ozet): {state['sagopa_verses'][0][:200] if state.get('sagopa_verses') else 'Henuz yok'}
Ceza'nin verse'i (ozet): {state['ceza_verses'][0][:200] if state.get('ceza_verses') else 'Henuz yok'}

Her iki sanatcinin tarzini birlestiren, akilda kalici bir nakarat yaz. 4-6 satir."""

    if prev_feedback:
        prompt += f"\n\n(Onceki nakarat reddedildi. Elestiri: {prev_feedback})"

    hook = await orchestrator_endpoint.generate(prompt,
        "Sen bir nakarat yazarisin. Verilen verse'lere ve temaya uygun nakarat yaz. Sadece nakarat satirlarini dondur.")
    print(f"  Hook:\n{hook}")
    return {"hook": hook}


async def critic_node(state: SongState) -> dict:
    """Verse'leri degerlendirir."""
    print("\n" + "=" * 60)
    print(f"ADIM 4 - VERSE CRITIC: Verse'ler degerlendiriliyor... (iterasyon {state.get('revision_count', 0) + 1})")
    print("=" * 60)

    all_sagopa = {}
    for i, verse in enumerate(state.get("sagopa_verses", [])):
        all_sagopa[f"sagopa_v{i+1}"] = verse
    all_ceza = {}
    for i, verse in enumerate(state.get("ceza_verses", [])):
        all_ceza[f"ceza_v{i+1}"] = verse

    all_verses = {**all_sagopa, **all_ceza}

    sagopa_context_text = "\n\n".join([f"[{k}]: {v}" for k, v in all_sagopa.items()])
    ceza_context_text = "\n\n".join([f"[{k}]: {v}" for k, v in all_ceza.items()])

    scores = {}
    feedback = {}
    for section_name, verse_text in all_verses.items():
        is_sagopa = "sagopa" in section_name
        artist_name = "Sagopa Kajmer" if is_sagopa else "Ceza"
        other_artist = "Ceza" if is_sagopa else "Sagopa Kajmer"
        other_context = ceza_context_text if is_sagopa else sagopa_context_text

        prompt = (
            f"Sanatci: {artist_name}\n"
            f"Bolum: {section_name}\n"
            f"Tema: {state['theme']}\n"
            f"Mood: {state['mood']}\n\n"
            f"Verse:\n{verse_text}\n\n"
            f"--- {other_artist}'nin verse'leri (uyum icin degerlendir) ---\n{other_context}\n\n"
            f"Bu verse'i degerlendir."
        )
        print(f"  [{section_name}] degerlendiriliyor...")
        response = await orchestrator_endpoint.generate(prompt, CRITIC_SYSTEM)
        evaluation = extract_json(response)
        if evaluation:
            raw = evaluation.get("score", 5)
            score = safe_score(raw)
            scores[section_name] = score
            if score < VERSE_PASS_THRESHOLD:
                feedback[section_name] = evaluation.get("feedback", "")
                print(f"  [{section_name}] REDDEDILDI (skor: {score}, ham: {raw})")
            else:
                print(f"  [{section_name}] ONAYLANDI (skor: {score}, ham: {raw})")
        else:
            scores[section_name] = 5.0
            feedback[section_name] = "Degerlendirme parse edilemedi."

    return {"critic_scores": scores, "critic_feedback": feedback, "revision_count": state.get("revision_count", 0) + 1}


async def hook_critic_node(state: SongState) -> dict:
    """Nakarati sarkinin geneli ile uyum acisindan degerlendirir."""
    hook_rev = state.get("hook_revision_count", 0)
    print("\n" + "=" * 60)
    print(f"ADIM 4b - HOOK CRITIC: Nakarat degerlendiriliyor... (hook iterasyon {hook_rev + 1})")
    print("=" * 60)

    hook_text = state.get("hook", "")
    all_verses_text = shared_memory.read_all()

    prompt = (
        f"Tema: {state['theme']}\n"
        f"Mood: {state['mood']}\n\n"
        f"NAKARAT:\n{hook_text}\n\n"
        f"--- SARKININ VERSE'LERI ---\n{all_verses_text}\n\n"
        f"Bu nakarati degerlendir."
    )
    print(f"  [hook] degerlendiriliyor...")
    response = await orchestrator_endpoint.generate(prompt, HOOK_CRITIC_SYSTEM)
    evaluation = extract_json(response)

    score = 5.0
    fb = ""
    if evaluation:
        raw = evaluation.get("score", 5)
        score = safe_score(raw)
        if score < HOOK_PASS_THRESHOLD:
            fb = evaluation.get("feedback", "")
            print(f"  [hook] REDDEDILDI (skor: {score}, ham: {raw})")
        else:
            print(f"  [hook] ONAYLANDI (skor: {score}, ham: {raw})")
    else:
        fb = "Degerlendirme parse edilemedi."

    return {
        "hook_critic_score": score,
        "hook_critic_feedback": fb,
        "hook_revision_count": hook_rev + 1,
    }


def should_revise(state: SongState) -> list:
    """Verse revizyon karari. Ikisi de yeniden yazar ama iyi verse'ler shared memory'de kalir."""
    scores = state.get("critic_scores", {})
    revision_count = state.get("revision_count", 0)

    if revision_count >= 10:
        print(f"  KARAR: Guvenlik siniri (10 iterasyon) ulasildi -> HOOK YAZIMI")
        return ["hook_writer"]

    if any(score < VERSE_PASS_THRESHOLD for score in scores.values()):
        low = {k: v for k, v in scores.items() if v < VERSE_PASS_THRESHOLD}
        high = [k for k, v in scores.items() if v >= VERSE_PASS_THRESHOLD]
        print(f"  KARAR: Dusuk skorlar {low} -> IKISI DE REVIZYON (iterasyon {revision_count})")
        # Iyi verse'leri shared memory'de koru - yeniden yazarken context olarak kullanilacak
        shared_memory.clear(keep_sections=high if high else None)
        return ["sagopa_writer", "ceza_writer"]

    print(f"  KARAR: Tum verse skorlar >= {VERSE_PASS_THRESHOLD} -> HOOK YAZIMI (toplam {revision_count} iterasyon)")
    return ["hook_writer"]


def should_revise_hook(state: SongState) -> str:
    """Hook revizyon karari."""
    score = state.get("hook_critic_score", 0)
    hook_rev = state.get("hook_revision_count", 0)

    if hook_rev >= 5:
        print(f"  KARAR: Hook guvenlik siniri (5 iterasyon) -> ASSEMBLY")
        return "assembly"

    if score < HOOK_PASS_THRESHOLD:
        print(f"  KARAR: Hook skor {score} < {HOOK_PASS_THRESHOLD} -> HOOK TEKRAR YAZILACAK (iterasyon {hook_rev})")
        return "hook_writer"

    print(f"  KARAR: Hook skor {score} >= {HOOK_PASS_THRESHOLD} -> ASSEMBLY (toplam {hook_rev} hook iterasyonu)")
    return "assembly"


async def assembly_node(state: SongState) -> dict:
    print("\n" + "=" * 60)
    print("ADIM 5 - ASSEMBLY: Sarki birlestiriliyor...")
    print("=" * 60)
    structure = state.get("song_structure", [])
    sagopa_idx = 0
    ceza_idx = 0
    song_parts = []
    for section in structure:
        if section == "intro":
            song_parts.append("[Intro]\n...")
        elif section.startswith("sagopa_"):
            if sagopa_idx < len(state.get("sagopa_verses", [])):
                song_parts.append(f"[Sagopa Kajmer - Verse {sagopa_idx + 1}]\n{state['sagopa_verses'][sagopa_idx]}")
                sagopa_idx += 1
        elif section.startswith("ceza_"):
            if ceza_idx < len(state.get("ceza_verses", [])):
                song_parts.append(f"[Ceza - Verse {ceza_idx + 1}]\n{state['ceza_verses'][ceza_idx]}")
                ceza_idx += 1
        elif "hook" in section:
            song_parts.append(f"[Nakarat]\n{state.get('hook', '')}")
        elif section == "outro":
            song_parts.append("[Outro]\n...")
    final_song_body = "\n\n".join(song_parts)

    title_prompt = f"Tema: {state['theme']}\nMood: {state['mood']}\n\nSarki sozleri:\n{final_song_body[:500]}\n\nBu sarkiya uygun bir isim sec."
    song_title = (await orchestrator_endpoint.generate(title_prompt, TITLE_SYSTEM)).strip().strip('"').strip("'")
    print(f"  Sarki ismi: {song_title}")

    md_song = f"""# {song_title}
**Sanatcilar:** Sagopa Kajmer ft. Ceza
**Tema:** {state['theme']}
**Mood:** {state['mood']}
**Toplam verse revizyon:** {state.get('revision_count', 0)}
**Toplam hook revizyon:** {state.get('hook_revision_count', 0)}

{final_song_body}
"""
    return {"song_title": song_title, "final_song": md_song, "status": "assembly_complete"}


async def beat_suggestion_node(state: SongState) -> dict:
    print("\n" + "=" * 60)
    print("ADIM 6 - BEAT SUGGESTION")
    print("=" * 60)
    prompt = f"Sarki temasi: {state['theme']}\nMood: {state['mood']}\n\nBu sarki icin genel bir beat onerisi olustur. SADECE JSON dondur."
    beat_sys = "Sen bir beat maker'sin. Kisa ve oz bir JSON dondur. Genel bir oneri ver."
    response = await orchestrator_endpoint.generate(prompt, beat_sys, max_tokens=512)
    suggestion = extract_json(response)
    if not suggestion:
        response = await orchestrator_endpoint.generate(prompt, beat_sys, max_tokens=512)
        suggestion = extract_json(response)
        if not suggestion:
            suggestion = {"bpm": 90, "key": "Am", "style": "boom bap"}
    print(f"  Beat: {json.dumps(suggestion, ensure_ascii=False, indent=2)}")
    return {"beat_suggestion": suggestion, "status": "completed"}


print("Node fonksiyonlari hazir!")

## 9. Graph Olustur ve Calistir

In [ ]:
def build_song_graph():
    """LangGraph pipeline'ini olusturur.

    Akis:
        ideation -> style_profiler -> producer
        -> [sagopa_writer, ceza_writer] (paralel, shared memory ile canli iletisim)
        -> verse_critic
            -> (herhangi biri < 7) -> [sagopa_writer, ceza_writer] (ikisi de yeniden yazar)
            -> (hepsi >= 7) -> hook_writer -> hook_critic
                -> (hook < 7) -> hook_writer (tekrar yaz)
                -> (hook >= 7) -> assembly -> beat_suggester -> END
    """
    graph = StateGraph(SongState)

    graph.add_node("ideation", ideation_node)
    graph.add_node("style_profiler", style_profiler_node)
    graph.add_node("producer", producer_node)
    graph.add_node("sagopa_writer", sagopa_node)
    graph.add_node("ceza_writer", ceza_node)
    graph.add_node("hook_writer", hook_node)
    graph.add_node("verse_critic", critic_node)
    graph.add_node("hook_critic", hook_critic_node)
    graph.add_node("assembly", assembly_node)
    graph.add_node("beat_suggester", beat_suggestion_node)

    # Ideation -> Style -> Producer
    graph.set_entry_point("ideation")
    graph.add_edge("ideation", "style_profiler")
    graph.add_edge("style_profiler", "producer")

    # Producer -> paralel yazim (fan-out, shared memory ile birbirini goruyor)
    graph.add_edge("producer", "sagopa_writer")
    graph.add_edge("producer", "ceza_writer")

    # Yazarlar -> Verse Critic (fan-in)
    graph.add_edge("sagopa_writer", "verse_critic")
    graph.add_edge("ceza_writer", "verse_critic")

    # Verse Critic -> conditional:
    #   Revizyon: ikisi de yeniden yazar (paralel fan-out)
    #   Onay: hook yazimina gec
    graph.add_conditional_edges("verse_critic", should_revise, {
        "sagopa_writer": "sagopa_writer",
        "ceza_writer": "ceza_writer",
        "hook_writer": "hook_writer",
    })

    # Hook -> Hook Critic
    graph.add_edge("hook_writer", "hook_critic")

    # Hook Critic -> conditional:
    #   Revizyon: hook tekrar yazilir
    #   Onay: assembly'ye gec
    graph.add_conditional_edges("hook_critic", should_revise_hook, {
        "hook_writer": "hook_writer",
        "assembly": "assembly",
    })

    # Assembly -> Beat -> END
    graph.add_edge("assembly", "beat_suggester")
    graph.add_edge("beat_suggester", END)

    return graph.compile()

print("Graph builder hazir!")

In [ ]:
# ============================================================
# PIPELINE CALISTIR
# ============================================================

async def create_song() -> dict:
    """Tam otonom sarki uretim pipeline'i."""
    app = build_song_graph()

    initial_state: SongState = {
        "theme": "",
        "mood": "",
        "additional_directives": "",
        "sagopa_style_prompt": "",
        "ceza_style_prompt": "",
        "song_structure": [],
        "section_briefs": {},
        "sagopa_verses": [],
        "ceza_verses": [],
        "hook": "",
        "critic_scores": {},
        "critic_feedback": {},
        "revision_count": 0,
        "hook_revision_count": 0,
        "hook_critic_score": 0.0,
        "hook_critic_feedback": "",
        "song_title": "",
        "final_song": "",
        "beat_suggestion": {},
        "status": "initialized",
    }

    # Shared memory'yi temizle (yeni sarki)
    shared_memory.clear()

    print("=" * 60)
    print("PIPELINE BASLATIILIYOR (tam otonom mod)")
    print("Kumru-2B + LoRA (Colab GPU) + OpenRouter Orchestrator")
    print("Shared Memory: AKTIF (paralel yazim sirasinda canli iletisim)")
    print(f"Verse esigi: {VERSE_PASS_THRESHOLD} | Hook esigi: {HOOK_PASS_THRESHOLD}")
    print("=" * 60)

    final_state = None
    async for event in app.astream(initial_state):
        for node_name, node_output in event.items():
            status = node_output.get("status", "")
            print(f"\n[{node_name}] tamamlandi.{f' Status: {status}' if status else ''}")
            if node_name == "verse_critic":
                scores = node_output.get("critic_scores", {})
                for section, score in scores.items():
                    mark = "PASS" if score >= VERSE_PASS_THRESHOLD else "FAIL"
                    print(f"  {mark} {section}: {score}/10")
            elif node_name == "hook_critic":
                score = node_output.get("hook_critic_score", 0)
                mark = "PASS" if score >= HOOK_PASS_THRESHOLD else "FAIL"
                print(f"  {mark} hook: {score}/10")
            final_state = {**initial_state, **node_output} if final_state is None else {**final_state, **node_output}

    print("\n" + "=" * 60)
    print("SARKI TAMAMLANDI!")
    print("=" * 60)
    return final_state or {}

# Calistir!
result = asyncio.run(create_song())

## 10. Sonuclari Goruntule

In [ ]:
from IPython.display import Markdown, display

if result and result.get("final_song"):
    display(Markdown(result["final_song"]))
else:
    print("Sarki uretilemedi.")

In [ ]:
# Beat onerisi
if result and result.get("beat_suggestion"):
    print("Beat Onerisi:")
    print(json.dumps(result["beat_suggestion"], ensure_ascii=False, indent=2))

In [ ]:
# Critic skorlari
if result:
    if result.get("critic_scores"):
        print("Verse Critic Skorlari:")
        for section, score in result["critic_scores"].items():
            mark = "PASS" if score >= VERSE_PASS_THRESHOLD else "FAIL"
            print(f"  [{mark}] {section}: {score}/10")
        print(f"\nToplam verse revizyon sayisi: {result.get('revision_count', 0)}")

    hook_score = result.get("hook_critic_score", 0)
    if hook_score:
        mark = "PASS" if hook_score >= HOOK_PASS_THRESHOLD else "FAIL"
        print(f"\nHook Critic Skoru:")
        print(f"  [{mark}] hook: {hook_score}/10")
        print(f"Toplam hook revizyon sayisi: {result.get('hook_revision_count', 0)}")